<a href="https://colab.research.google.com/github/jkranyak/project_3/blob/main/experimental_models/gan_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install isic-cli
!pip install isic-cli
!pip install kaggle
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.7 MB/s eta 0:00:00


In [2]:
! isic user login

visit the following url in a browser:
https://api.isic-archive.com/oauth/authorize?response_type=code&client_id=RpCzc4hFjv5gOJdM2DM2nBdokOviOh5ne63Tpn7Q&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=rooV5mKCOjQfpCbwasKeabK6zSoHcH&code_challenge=kszlN5OaPWsSBH1VVogXxhubn8IFM7KpnAvyH1lF7tU&code_challenge_method=S256
enter the code shown in your browser: jqFbNnuMOBUhdaFrgfkxFppfCnWCgO
Success!


In [3]:
!isic collection list

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ ID  ┃ Name                                          ┃ Public ┃ Pinned ┃ Locked ┃ DOI             ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 249 │ BCN20000                                      │ True   │ False  │ False  │ None            │
│ 61  │ Challenge 2016: Test                          │ True   │ True   │ True   │ None            │
│ 74  │ Challenge 2016: Training                      │ True   │ True   │ True   │ None            │
│ 69  │ Challenge 2017: Test                          │ True   │ True   │ True   │ None            │
│ 60  │ Challenge 2017: Training                      │ True   │ True   │ True   │ None            │
│ 71  │ Challenge 2017: Validation                    │ True   │ True   │ True   │ None            │
│ 64  │ Challenge 2018: Task 1-2: Test                │ True   │ True   │ True   │ None    

Step 2: Query the Dataset
Endpoints: Use the API endpoints to query the dataset. Common operations include listing available images, retrieving image metadata, and downloading images.
Filtering: Utilize query parameters to filter the dataset based on your criteria, such as diagnosis, image type, or other metadata.

In [4]:
import requests
import os
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from imblearn.over_sampling import SMOTE
from pathlib import Path
from google.colab import drive
from PIL import Image

In [6]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Set up directory path for saving images
image_dir = "/content/drive/My Drive/Generated_Images"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

Mounted at /content/drive


In [7]:
def load_real_images(num_images):
    # Placeholder for loading real images for evaluation
    # You need to implement this method to load 'num_images' real images from your dataset
    pass

Comment out if already running

In [8]:
# Specify the current path of kaggle.json file
current_path = '/content/kaggle.json'

# Desired path where the Kaggle API expects the kaggle.json file
desired_path = '/root/.kaggle/kaggle.json'

if os.path.exists(current_path):
    os.makedirs(os.path.dirname(desired_path), exist_ok=True)
    os.rename(current_path, desired_path)

    # Set the file's permissions to avoid a permissions error
    os.chmod(desired_path, 0o600)
else:
    print(f"Error: '{current_path}' does not exist. Please upload the file.")



find the data set on kaggle

Comment out if running and already have downloaded images

In [9]:
!kaggle datasets download -d andrewmvd/isic-2019


100% 9.09G/9.10G [01:31<00:00, 83.0MB/s]
100% 9.10G/9.10G [01:31<00:00, 106MB/s] 


In [10]:
!unzip -q isic-2019.zip


Step 1: Data Preparation and Preprocessing
DICOM Images





Load the metadata

and

Explore the Metadata CSV: Load the metadata.csv files for training, test, and validation sets to understand the structure and types of data available. This step is crucial for preprocessing and feature selection.

In [11]:
# Load the metadata
metadata = pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

# Display the first few rows of each DataFrame
metadata.head()


,image,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,55.0,anterior torso,NaN,female
1,ISIC_0000001,30.0,anterior torso,NaN,female
2,ISIC_0000002,60.0,upper extremity,NaN,female
3,ISIC_0000003,30.0,upper extremity,NaN,male
4,ISIC_0000004,80.0,posterior torso,NaN,male


In [12]:
ground_truth = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
ground_truth

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Load the CSV files
ground_truth = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
metadata = pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

image_dir = Path('/content/ISIC_2019_Training_Images')
ground_truth['image_path'] = ground_truth['image'].apply(lambda x: image_dir / f"{x}.jpg")

# Merge the ground_truth with metadata if necessary
full_metadata = pd.merge(ground_truth, metadata, on='image', how='left')  # Adjust 'on' parameter as needed
full_metadata

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,image_path,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,55.0,anterior torso,NaN,female
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,30.0,anterior torso,NaN,female
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,60.0,upper extremity,NaN,female
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,30.0,upper extremity,NaN,male
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,80.0,posterior torso,NaN,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,85.0,head/neck,BCN_0003925,female
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,65.0,anterior torso,BCN_0001819,male
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,70.0,lower extremity,BCN_0001085,male
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007325...,55.0,palms/soles,BCN_0002083,female


In [14]:
# Display the image paths for the first few entries
print(full_metadata['image_path'].head())


0    /content/ISIC_2019_Training_Images/ISIC_000000...
1    /content/ISIC_2019_Training_Images/ISIC_000000...
2    /content/ISIC_2019_Training_Images/ISIC_000000...
3    /content/ISIC_2019_Training_Images/ISIC_000000...
4    /content/ISIC_2019_Training_Images/ISIC_000000...
Name: image_path, dtype: object


In [15]:
# Correct the base path in 'image_path' column
correct_base_path = "/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input"

full_metadata['image_path'] = full_metadata['image'].apply(lambda x: f"{correct_base_path}/{x}.jpg")

# Verify the correction by printing the first few entries again
print(full_metadata['image_path'].head())


0    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
1    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
2    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
3    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
4    /content/ISIC_2019_Training_Input/ISIC_2019_Tr...
Name: image_path, dtype: object


In [61]:
full_metadata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25331 entries, 0 to 25330
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   image                                25331 non-null  object 
 1   MEL                                  25331 non-null  float64
 2   NV                                   25331 non-null  float64
 3   BCC                                  25331 non-null  float64
 4   AK                                   25331 non-null  float64
 5   BKL                                  25331 non-null  float64
 6   DF                                   25331 non-null  float64
 7   VASC                                 25331 non-null  float64
 8   SCC                                  25331 non-null  float64
 9   UNK                                  25331 non-null  float64
 10  image_path                           25331 non-null  object 
 11  age_approx                  

In [17]:
# Fill missing values for 'age_approx' with its median
full_metadata['age_approx'].fillna(full_metadata['age_approx'].median(), inplace=True)

# For categorical data, fill missing values with 'unknown'
full_metadata['anatom_site_general'].fillna('unknown', inplace=True)
full_metadata['sex'].fillna('unknown', inplace=True)


In [18]:
full_metadata = pd.get_dummies(full_metadata, columns=['anatom_site_general', 'sex'])


In [62]:
import pandas as pd

# Load your full metadata if not already loaded
# full_metadata = pd.read_csv('path_to_metadata.csv')

# Calculate the sum for each class to see the distribution
class_distribution = full_metadata[['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']].sum()
print("Class Distribution:\n", class_distribution)

# Identify underrepresented classes based on a threshold
threshold = 100  # Set this based on your specific needs or observations
underrepresented_classes = class_distribution[class_distribution < threshold].index
print("Underrepresented Classes:", underrepresented_classes)


Class Distribution:
 MEL      4522.0
NV      12875.0
BCC      3323.0
AK        867.0
BKL      2624.0
DF        239.0
VASC      253.0
SCC       628.0
dtype: float64
Underrepresented Classes: Index([], dtype='object')


#Define the Generator and Discriminator Networks



In [21]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Multiply, BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# Assuming latent dimension and number of unique classes
latent_dim = 100  # Common choice for image data
num_classes = 9   # Adjust based on your specific dataset

# Build the generator model
def build_generator(latent_dim, num_classes):
    model = Sequential()
    model.add(Dense(128 * 56 * 56, activation="relu", input_dim=latent_dim))  # Adjust size here
    model.add(Reshape((56, 56, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(3, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))
    model_input = Multiply()([noise, label_embedding])
    img = model(model_input)

    return Model([noise, label], img)

generator = build_generator(latent_dim, num_classes)
generator.summary()

# Build the discriminator model
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 100)               900       ['input_4[0][0]']             
                                                                                                  
 input_3 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 flatten_1 (Flatten)         (None, 100)                  0         ['embedding_1[0][0]']         
                                                                                            

#Define the Loss Functions

In [22]:
def train(generator, discriminator, combined, X_train, epochs, batch_size, latent_dim, num_classes, class_proportions, save_interval):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Number of batches
        num_batches = int(X_train.shape[0] / batch_size)

        for batch in range(num_batches):
            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and categorical labels
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            sampled_labels = np.random.choice(range(num_classes), batch_size, p=class_proportions)

            # Generate a batch of new images
            gen_imgs = generator.predict([noise, sampled_labels])

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = discriminator.train_on_batch(imgs, valid)
            d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = combined.train_on_batch([noise, sampled_labels], valid)

        # Output training status
        print(f"Epoch: {epoch}, D loss: {d_loss}, G loss: {g_loss}")

        # If at save interval => save generated image samples
        if (epoch + 1) % save_interval == 0:
            save_images(epoch, generator, sampled_labels)



In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt

def save_images(epoch, generator, labels, image_dir, n_classes=10, latent_dim=100):
    noise = np.random.normal(0, 1, (n_classes * 10, latent_dim))
    sampled_labels = np.array([i for i in range(n_classes) for _ in range(10)])
    gen_imgs = generator.predict([noise, sampled_labels])
    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale images

    fig, axs = plt.subplots(n_classes, 10, figsize=(10, n_classes), dpi=100)
    cnt = 0
    for i in range(n_classes):
        for j in range(10):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :])
            axs[i, j].axis('off')
            cnt += 1
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    fig.savefig(f"{image_dir}/gan_generated_epoch_{epoch}.png")
    plt.close()

def train(generator, discriminator, combined, X_train, epochs, batch_size=128, latent_dim=100, num_classes=10, save_interval=50, image_dir='/path/to/save'):
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        sampled_labels = np.random.randint(0, num_classes, batch_size)
        gen_imgs = generator.predict([noise, sampled_labels])

        d_loss_real = discriminator.train_on_batch(imgs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = combined.train_on_batch([noise, sampled_labels], np.ones((batch_size, 1)))

        if epoch % save_interval == 0:
            save_images(epoch, generator, sampled_labels, image_dir, n_classes=num_classes, latent_dim=latent_dim)

        if epoch % 10 == 0 or epoch == epochs - 1:
            print(f"Epoch: {epoch}, D loss: {d_loss:.4f}, G loss: {g_loss:.4f}")


# Make sure to define `latent_dim`, `num_classes`, `img_shape`, `X_train`, etc. before running the training.



#Compile the Model

In [24]:
# Assuming latent dimension and number of classes
latent_dim = 100
num_classes = 10
img_shape = (224, 224, 3)

# Instantiate and build the generator and discriminator
generator = build_generator(latent_dim, num_classes)
discriminator = build_discriminator(img_shape)

# Print model summaries for verification
print("Generator Summary:")
generator.summary()
print("\nDiscriminator Summary:")
discriminator.summary()

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(0.0002, 0.5),
                      metrics=['accuracy'])

# Setup for combined model
discriminator.trainable = False
z = Input(shape=(latent_dim,))
label = Input(shape=(1,))
img = generator([z, label])
valid = discriminator(img)
combined = Model([z, label], valid)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Confirm models are built and compiled
print("\nCombined Model Summary:")
combined.summary()
print("\nGenerator and Discriminator models are built and compiled.")


Generator Summary:
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 1, 100)               1000      ['input_6[0][0]']             
                                                                                                  
 input_5 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 flatten_2 (Flatten)         (None, 100)                  0         ['embedding_2[0][0]']         
                                                                         

#Training Loop:

In [28]:
test_noise = np.random.normal(0, 1, (100, latent_dim))


In [29]:
test_labels = np.array([num for num in range(num_classes) for _ in range(10)])


In [30]:
print(f"Shape of test_noise: {test_noise.shape}")
print(f"Shape of test_labels: {test_labels.shape}")


Shape of test_noise: (100, 100)
Shape of test_labels: (90,)


In [68]:
# Define class label mapping based on your specific classes and their indices
class_label_mapping = {
    'MEL': 0,  # Melanoma
    'NV': 1,   # Melanocytic nevus
    'BCC': 2,  # Basal cell carcinoma
    'AK': 3,   # Actinic keratosis
    'BKL': 4,  # Benign keratosis
    'DF': 5,   # Dermatofibroma
    'VASC': 6,  # Vascular lesion
    'SCC': 7   # Squamous cell carcinoma
}

# Update the usage in your loop to reflect this mapping
underrepresented_classes = ['DF', 'VASC', 'AK', 'SCC']
additional_images_needed = {'DF': 300, 'VASC': 300, 'AK': 500, 'SCC': 300}

In [69]:
# Analyze class distribution and identify underrepresented classes
class_distribution = full_metadata[['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']].sum()
print("Class Distribution:\n", class_distribution)

# Assuming class_label_mapping and generator are already defined
underrepresented_classes = ['DF', 'VASC', 'AK', 'SCC']
additional_images_needed = {'DF': 300, 'VASC': 300, 'AK': 500, 'SCC': 300}


Class Distribution:
 MEL      4522.0
NV      12875.0
BCC      3323.0
AK        867.0
BKL      2624.0
DF        239.0
VASC      253.0
SCC       628.0
dtype: float64


In [92]:
import matplotlib.pyplot as plt
import numpy as np

def generate_and_save_images(generator, epoch, test_input, test_labels, image_dir='/content/myimages/training'):
    # Ensure the directory exists before saving
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

    # Generate predictions using the model
    predictions = generator.predict([test_input, test_labels])  # Ensure the input is correctly shaped

    # Save each image individually
    for i, image_array in enumerate(predictions):
        # Normalize the image data to 0-255
        image_array = ((image_array + 1) * 127.5).astype(np.uint8)
        # Save the image
        image_path = os.path.join(image_dir, f"image_epoch_{epoch}_class_{test_labels[i]}_num_{i:04d}.png")
        Image.fromarray(image_array).save(image_path)

    print(f"Images saved in {image_dir}")

latent_dim = 100  # Ensure this matches your GAN setup
num_classes = 8

# Assuming test_labels needs to be adjusted based on the underrepresented classes needing generation
class_label_index = class_label_mapping[class_label]  # Get the numeric index for the GAN

# Generate test noise and labels
test_noise = np.random.normal(0, 1, (num_images, latent_dim))
test_labels = np.full((num_images,), class_label_index)

# Call the function correctly
generate_and_save_images(generator, epoch, test_noise, test_labels, '/content/myimages/training')

10/10 [==============================] - 1s 66ms/step
Images saved in /content/myimages/training


In [109]:
# Import necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

# Assuming generator is already built and compiled
# Assuming class_label_mapping is correctly defined

# Define the epoch for naming files (if needed)
epoch = 0
image_dir = '/content/myimages/training'

for class_label, num_images in additional_images_needed.items():
    if class_label in underrepresented_classes:
        print(f"Generating {num_images} images for class {class_label}")
        class_index = class_label_mapping[class_label]

        # Generate noise and labels
        test_noise = np.random.normal(0, 1, (num_images, latent_dim))
        test_labels = np.full((num_images,), class_index)

        # Call the function with correctly ordered parameters
        generate_and_save_images(generator, epoch, test_noise, test_labels, image_dir)

        # Prepare new entries for metadata
        new_entries = pd.DataFrame({
            'image_id': [f'{class_label}_generated_{i}' for i in range(num_images)],
            class_label: 1,
            **{cl: 0 for cl in class_label_mapping if cl != class_label}
        })

        # Concatenate new entries with the existing metadata
        full_metadata = pd.concat([full_metadata, new_entries], ignore_index=True)

# Save updated metadata to CSV
full_metadata.to_csv('updated_full_metadata.csv', index=False)


Generating 300 images for class DF
10/10 [==============================] - 1s 67ms/step
Images saved in /content/myimages/training
Generating 300 images for class VASC
10/10 [==============================] - 1s 66ms/step
Images saved in /content/myimages/training
Generating 500 images for class AK
16/16 [==============================] - 1s 67ms/step
Images saved in /content/myimages/training
Generating 300 images for class SCC
10/10 [==============================] - 1s 66ms/step
Images saved in /content/myimages/training


In [100]:
from PIL import Image
import numpy as np
import os

def load_real_images(num_images, directory, target_size=(299, 299), normalize=False):
    if not os.path.exists(directory):
        raise FileNotFoundError(f"The specified directory {directory} does not exist.")

    all_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.jpg')]
    print(f"Total images available: {len(all_files)}")  # Debugging statement

    if len(all_files) < num_images:
        print(f"Warning: Only {len(all_files)} images found, which is less than the requested {num_images}. Adjusting to available images.")
        num_images = len(all_files)  # Adjust to use all available images

    selected_files = np.random.choice(all_files, num_images, replace=False)
    images = []
    for file in selected_files:
        img = Image.open(file).resize(target_size)
        if normalize:
            img = np.array(img).astype('float32') / 255.0
        else:
            img = np.array(img)
        images.append(img)

    return np.array(images)


In [107]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import numpy as np
from scipy.linalg import sqrtm

def calculate_fid(model, real_images, generated_images):
    if real_images.shape != generated_images.shape:
        raise ValueError("Shape mismatch between real and generated images")

    # Preprocess the images as InceptionV3 expects inputs normalized
    real_images = preprocess_input(real_images)
    generated_images = preprocess_input(generated_images)

    # Get activations from InceptionV3
    act1 = model.predict(real_images)
    act2 = model.predict(generated_images)

    # Compute mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    # Calculate the sum of squared differences between the means
    ssdiff = np.sum((mu1 - mu2)**2.0)

    # Compute sqrt of the product of covariance matrices to handle potential numerical instability
    covmean = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):  # Check for complex numbers
        covmean = covmean.real

    # Calculate FID score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid
p

NameError: name 'real_images' is not defined

In [106]:
import numpy as np

def evaluate_generator(generator, model, num_images=100, latent_dim=100, num_classes=10, normalize=False):
    # Generate noise and labels according to the generator's expected input dimensions and label count
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    sampled_labels = np.array([i % num_classes for i in range(num_images)])
    generated_images = generator.predict([noise, sampled_labels])

    # Directory where the real images are stored
    directory = '/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input'

    # Load real images, ensuring we have enough
    real_images = load_real_images(num_images, directory)
    if real_images is None:
        raise ValueError("Not enough real images available to perform evaluation.")

    # Calculate FID score using the provided model, likely an InceptionV3 model
    fid_value = calculate_fid(model, real_images, generated_images)
    print(f'FID score: {fid_value}')

# Example usage
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Load or define your InceptionV3 model for FID calculation
# Ensure it is pre-loaded with the correct weights and include the preprocessing function
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

# Evaluate the generator
evaluate_generator(generator, inception_model, num_images=100, latent_dim=100, num_classes=10)


4/4 [==============================] - 0s 65ms/step
Total images available: 25331


ValueError: Shape mismatch between real and generated images

In [104]:
print("Shape of Real Images:", real_images.shape)
print("Shape of Generated Images:", generated_images.shape)


NameError: name 'real_images' is not defined